### Testing 5 different transformer models with 3 seeds each
### Task: Argument Stance Classification (Support/Oppose)

In [8]:
# Libraries
import os
import numpy as np
import pandas as pd
from datasets import Dataset, load_from_disk
from transformers import AutoTokenizer, AutoModelForSequenceClassification, TrainingArguments, Trainer, EarlyStoppingCallback
from sklearn.metrics import accuracy_score, f1_score, precision_score, recall_score
import torch
from tqdm.auto import tqdm
import warnings
warnings.filterwarnings('ignore')

# Google Colab or not
try:
    import google.colab
    IN_COLAB = True
except:
    IN_COLAB = False

if IN_COLAB:
    from google.colab import drive
    drive.mount('/content/drive')
    path = "/content/drive/MyDrive/multimodal-argmining"
    os.chdir(path)
    print(f"Loading data from Google Drive: {path}")
else:
    path = "C:/Users/diego/Desktop/Master Neuro/M2/Intership_NLP/multimodal-argmining"
    os.chdir(path)
    print(f"Loading data locally from: {path}")


# GPU
if torch.cuda.is_available():
    device = torch.device("cuda")
    print("GPU ready:", torch.cuda.get_device_name(0))
else:
    device = torch.device("cpu")
    print("No GPU detecting, using CPU.")

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).
Loading data from Google Drive: /content/drive/MyDrive/multimodal-argmining
GPU ready: Tesla T4


In [9]:
# WE define our Models to tests and the seeds
MODELS = [
    "roberta-base",
    "microsoft/deberta-v3-base",
    "cardiffnlp/twitter-roberta-base",
    "bert-base-uncased",
    "microsoft/deberta-base",
    "ddore14/RooseBERT-scr-uncased"
]

#Seeds
SEEDS = [42, 123, 456]


print("\nModels to test:")
for i, model in enumerate(MODELS, 1):
    print(f"  {i}. {model}")



Models to test:
  1. roberta-base
  2. microsoft/deberta-v3-base
  3. cardiffnlp/twitter-roberta-base
  4. bert-base-uncased
  5. microsoft/deberta-base
  6. ddore14/RooseBERT-scr-uncased


In [10]:
#Load Dataset
train_path = f"{path}/data/gun_control_train.csv"
dev_path   = f"{path}/data/gun_control_dev.csv"
test_path  = f"{path}/data/gun_control_test.csv"

df_train = pd.read_csv(train_path)
df_dev   = pd.read_csv(dev_path)
df_test  = pd.read_csv(test_path)


# Map labels to ints
label2id = {"oppose": 0, "support": 1}
for df in [df_train, df_dev, df_test]:
    df["label"] = df["stance"].map(label2id)

print(df_train["label"].value_counts())
df_train.head()



dataset_train = Dataset.from_pandas(df_train[["tweet_text", "label"]])
dataset_dev   = Dataset.from_pandas(df_dev[["tweet_text", "label"]])
dataset_test  = Dataset.from_pandas(df_test[["tweet_text", "label"]])


label
1    475
0    448
Name: count, dtype: int64


In [11]:

# Tokenization Function for each model
def tokenize_dataset(dataset, tokenizer, max_length=128):

    def tokenize_batch(batch):
        return tokenizer(batch["tweet_text"],padding="max_length",truncation=True,max_length=max_length)

    tokenized = dataset.map(tokenize_batch, batched=True)

    tokenized.set_format(type='torch', columns=['input_ids', 'attention_mask', 'label'])

    return tokenized


In [12]:
# We define metrics Function
def compute_metrics(pred):
    labels = pred.label_ids
    preds = np.argmax(pred.predictions, axis=1)

    acc = accuracy_score(labels, preds)
    f1 = f1_score(labels, preds, average="weighted")
    precision = precision_score(labels, preds, average="weighted")
    recall = recall_score(labels, preds, average="weighted")

    return {
        "accuracy": acc,
        "f1": f1,
        "precision": precision,
        "recall": recall,
    }

In [13]:
# Training Function
def train_and_evaluate(model_name, seed, train_dataset, dev_dataset,test_dataset):

    # Set seed!
    np.random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

    print(f"Training: {model_name} | Seed: {seed}")
    print(f"{'='*60}")

    # Load tokenizer
    tokenizer = AutoTokenizer.from_pretrained(model_name)
    print(f"Tokenizer loaded for {model_name}...")

    # Tokenize datasets with model tokenizer
    train_dataset_tok = tokenize_dataset(train_dataset, tokenizer, 128)
    dev_dataset_tok = tokenize_dataset(dev_dataset, tokenizer, 128)
    test_dataset_tok = tokenize_dataset(test_dataset, tokenizer, 128)
    print(f"Tokenization complete")


    # Load model
    model = AutoModelForSequenceClassification.from_pretrained(model_name, num_labels=2)
    print(f"Model Loaded: {model_name}.")


    # Training arguments
    training_args = TrainingArguments(
        output_dir=f"./temp_models/{model_name.replace('/', '_')}_seed{seed}",
        eval_strategy="epoch",
        save_strategy="epoch",
        logging_strategy="epoch",
        learning_rate=2e-5,
        per_device_train_batch_size=16,
        per_device_eval_batch_size=16,
        num_train_epochs=5,
        weight_decay=0.01,
        load_best_model_at_end=True,
        metric_for_best_model="f1",
        save_total_limit=1,
        report_to="none",
        logging_steps=10
    )

    # Initialize Trainer
    trainer = Trainer(
        model=model,
        args=training_args,
        train_dataset=train_dataset_tok,
        eval_dataset=dev_dataset_tok,
        processing_class=tokenizer,
        compute_metrics=compute_metrics,
        callbacks=[EarlyStoppingCallback(early_stopping_patience=2)],
    )

    # Train
    print(f"\n Starting training...")
    trainer.train()


    # Evaluate
    print(f"\n Evaluating on test set...")
    eval_results = trainer.evaluate(test_dataset_tok)

    # Extract metrics
    results = {
        "model": model_name,
        "seed": seed,
        "accuracy": eval_results["eval_accuracy"],
        "precision": eval_results["eval_precision"],
        "recall": eval_results["eval_recall"],
        "f1": eval_results["eval_f1"],
        "loss": eval_results["eval_loss"]
    }

    print(f"\n Results for {model_name} (seed {seed}):")
    print(f"   Accuracy:  {results['accuracy']:.4f}")
    print(f"   Precision: {results['precision']:.4f}")
    print(f"   Recall:    {results['recall']:.4f}")
    print(f"   F1-Score:  {results['f1']:.4f}")

    # Clean up to save memory
    del model
    del trainer
    del train_dataset_tok
    del dev_dataset_tok
    del test_dataset_tok

    torch.cuda.empty_cache() if torch.cuda.is_available() else None

    return results

In [14]:
# We run the Experiments
print("STARTING EXPERIMENT")
print("="*60)

all_results = []
total_experiments = len(MODELS) * len(SEEDS)
current_experiment = 0

for model_name in MODELS:
    print(f"# MODEL: {model_name}")

    for seed in SEEDS:
        current_experiment += 1
        print(f"\n[Experiment {current_experiment}/{total_experiments}]")
        results = train_and_evaluate(
            model_name=model_name,
            seed=seed,
            train_dataset=dataset_train,
            dev_dataset=dataset_dev,
            test_dataset=dataset_test)


        all_results.append(results)


print("EXPERIMENT COMPLETED")

STARTING EXPERIMENT
# MODEL: roberta-base

[Experiment 1/18]
Training: roberta-base | Seed: 42
Tokenizer loaded for roberta-base...


Map:   0%|          | 0/923 [00:00<?, ? examples/s]

Map:   0%|          | 0/100 [00:00<?, ? examples/s]

Map:   0%|          | 0/150 [00:00<?, ? examples/s]

Tokenization complete


...

Some weights of BertForSequenceClassification were not initialized from the model checkpoint at ddore14/RooseBERT-scr-uncased and are newly initialized: ['bert.pooler.dense.bias', 'bert.pooler.dense.weight', 'classifier.bias', 'classifier.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


Model Loaded: ddore14/RooseBERT-scr-uncased.

 Starting training...


Epoch,Training Loss,Validation Loss,Accuracy,F1,Precision,Recall
1,0.546100,0.397963,0.840000,0.837939,0.881290,0.840000
2,0.282100,0.278883,0.860000,0.860169,0.861026,0.860000
3,0.179200,0.286029,0.880000,0.880145,0.880994,0.880000
4,0.094200,0.336509,0.880000,0.880145,0.880994,0.880000
5,0.064000,0.350108,0.880000,0.880192,0.883200,0.880000



 Evaluating on test set...



 Results for ddore14/RooseBERT-scr-uncased (seed 456):
   Accuracy:  0.7867
   Precision: 0.7979
   Recall:    0.7867
   F1-Score:  0.7797
EXPERIMENT COMPLETED


In [15]:
# Results
results_df = pd.DataFrame(all_results)

# Per model we calculate the mean and std on each metric
model_stats = results_df.groupby('model').agg({
    'accuracy': ['mean', 'std'],
    'precision': ['mean', 'std'],
    'recall': ['mean', 'std'],
    'f1': ['mean', 'std']
}).round(4)

# Format
model_stats.columns = ['_'.join(col).strip() for col in model_stats.columns.values]
model_stats = model_stats.reset_index()
model_stats.columns = [
    'Model',
    'Accuracy_Mean', 'Accuracy_Std',
    'Precision_Mean', 'Precision_Std',
    'Recall_Mean', 'Recall_Std',
    'F1_Mean', 'F1_Std'
]

# Sort by F1 score
model_stats = model_stats.sort_values('F1_Mean', ascending=False).reset_index(drop=True)
print(model_stats)


                             Model  Accuracy_Mean  Accuracy_Std  \
0        microsoft/deberta-v3-base         0.8467        0.0200   
1                     roberta-base         0.8244        0.0038   
2           microsoft/deberta-base         0.8222        0.0038   
3  cardiffnlp/twitter-roberta-base         0.8200        0.0115   
4                bert-base-uncased         0.8022        0.0139   
5    ddore14/RooseBERT-scr-uncased         0.7956        0.0102   

   Precision_Mean  Precision_Std  Recall_Mean  Recall_Std  F1_Mean  F1_Std  
0          0.8549         0.0148       0.8467      0.0200   0.8434  0.0219  
1          0.8265         0.0041       0.8244      0.0038   0.8223  0.0039  
2          0.8347         0.0022       0.8222      0.0038   0.8170  0.0050  
3          0.8275         0.0142       0.8200      0.0115   0.8161  0.0115  
4          0.8039         0.0157       0.8022      0.0139   0.7998  0.0135  
5          0.8077         0.0101       0.7956      0.0102   0.7889  

In [16]:

# Results to CSV
os.makedirs(f"{path}/experiments/text/Performance/", exist_ok=True)
output_file = f"{path}/experiments/text/Performance/model_comparison_results.csv"
model_stats.to_csv(output_file, index=False)
print(f"\nSummary results saved to: {output_file}")


Summary results saved to: /content/drive/MyDrive/multimodal-argmining/experiments/text/Performance/model_comparison_results.csv
